# Question 1: Supervised Learning for Bioprinting

**Background**: Bioprinting uses additive manufacturing to create three-dimensional structures containing living cells, enabling controlled in vitro experiments that better mimic the tumour microenvironment. Certain printing parameters, such as extrusion pressure and nozzle diameter, can substantially reduce post-printing cell viability. Recent work has shown that supervised machine learning models can predict printing outcomes, including cell viability, from printer and bioink parameters. In *Machine Assisted Experimentation of Extrusion-Based Bioprinting Systems*, Tian et al. compiled a multi-laboratory dataset of 617 cell viability measurements and applied several ML models to predict printing outcomes.

**Objective**: In this question, you will partially reproduce and extend the supervised learning analysis of Tian et al. You will preprocess the experimental dataset and train two classification models to
predict whether a given set of bioprinting parameters results in acceptable cell viability (Yes/No). Assume that cell viability is considered acceptable if it exceeds the threshold defined in the paper.

### (1a) Written Long Answer

##### 1. Define and explain precision and recall in the context of binary classification. Provide and explain the formula for each metric.

- Precision is a metric that measures the accuracy of positive predictions ("Of the positive predictions, how many were correctly positive?")

        Precision = TP / (TP + FP), where TP is the amount of true positives and FP is the amount of false positives
- Recall is a metric that measures the model's ability to correctly classify positives ("How many positive predictions were correctly labelled?")

        Recall = TP / (TP + FN), where TP is the amount of true positives and FN is the amount of false negatives

##### 2. Why can accuracy alone be misleading when evaluating classification models? Explain why precision and recall are important complementary metrics, and give a concrete example.

Accuracy alone can be misleading when one class appears infrequently. For example, for a dataset where class A appears 1% of the time, we can apply a model that predicts class B 100% of the time. This would result in an accuracy of 100%, but of course this model is effectively useless. This is why precision and recall are important metrics to use as they tell you about false positives and false negatives. For example, a model may want to avoid false negatives as much as possible, like in bank fraud prediction, as the consequence of a false negative is extremely dire.

### (1b) Data Preprocessing

In [5]:
import pandas as pd

ds = pd.read_csv("Cell viability and extrusion dataset V1.csv")
ds.head()

,Reference,DOI,Final_Alginate_Conc_(%w/v),Final_Gelatin_Conc_(%w/v),Final_GelMA_Conc_(%w/v),Final_Hyaluronic_Acid_Conc_(%w/v),Final_MeHA_Conc_(%w/v),Final_NorHA_Conc_(%w/v),Final_Fibroin/Fibrinogen_Conc_(%w/v),Final_P127_Conc_(%w/v),...,Saline_Solution_Used?,EtOH_Solution_Used?,Photoinitiator_Used?,Enzymatic_Crosslinker_Used?,Matrigel_Used?,Conical_or_Straight_Nozzle,Primary/Not_Primary,Viability_at_time_of_observation_(%),Acceptable_Viability_(Yes/No),Acceptable_Pressure_(Yes/No)
0,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,96.0,Y,Y
1,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,72.0,N,N
2,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,80.0,Y,Y
3,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,0.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,96.0,Y,Y
4,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,0.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,77.0,N,N


- Handling missing values for bioink temperature as described in the paper